In [1]:
!pip install madgrad
!pip install transformers
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for madgrad: filename=madgrad-1.3-py3-none-any.whl size=11867 sha256=4a1e79542c7e78d24b866fd2c373bf94738256af9edeb2a060e37a2606573c6a
  Stored in directory: /root/.cache/pip/wheels/d9/a3/83/7ed1ddc517cd87cad4e3a4aec7f8ea1d5e83a5ff282e51490a
Successfully built madgrad
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.6 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-rwzv1udn
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-buil

In [2]:
# Standard libraries
import os
import json
import random
import copy
from collections import Counter

# Third-party libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from PIL import Image
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm

# Torch specific
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from madgrad import MADGRAD

# HuggingFace Transformers
import transformers
from transformers import (
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    MMBTConfig,
    MMBTModel,
    MMBTForClassification,
    get_linear_schedule_with_warmup,
)

# Other specific modules/packages
import clip
import pickle


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [4]:
# Load the CLIP model ("RN50x4" variant) and its preprocessing tools.
# The model is loaded onto the specified device (e.g., 'cuda' or 'cpu').
# The 'jit' argument determines whether the model should be loaded using PyTorch's Just-In-Time compiler; we've set it to False here.
clip_model, preprocess = clip.load("RN50x4", device=device, jit=False)

# Freeze all the weights of the CLIP model.
# This means the CLIP model will act as a feature extractor and won't be fine-tuned during training.
for p in clip_model.parameters():
    p.requires_grad = False

# Set the number of image embeddings.
num_image_embeds = 4

# Number of labels for classification. It's set to 1, possibly indicating a binary classification task.
num_labels = 1

# Gradient accumulation steps indicate how often we'll update model weights.
# For instance, with a value of 20, we'll backpropagate the gradient but only update the weights every 20 batches.
gradient_accumulation_steps = 20

# Directory where the data is stored.
data_dir = '/content/drive/MyDrive/cs688project2/data/'

# Set the maximum sequence length for tokenizing text inputs.
max_seq_length = 80

# Maximum gradient norm for gradient clipping. This helps in preventing extremely large gradient updates.
max_grad_norm = 0.5

# Training and evaluation batch sizes.
train_batch_size = 16
eval_batch_size = 16

# Define the size of the image encoder and the size of the image features.
image_encoder_size = 288
image_features_size = 640

# Number of epochs the model will be trained for.
num_train_epochs = 5


100%|███████████████████████████████████████| 402M/402M [00:09<00:00, 46.7MiB/s]


In [5]:
def slice_image(im, desired_size):
    """
    Resize the input image and slice it into three parts (left, center, right or top, center, bottom).

    Parameters:
    - im (PIL.Image): Input image.
    - desired_size (int): Desired size for slices.

    Returns:
    - list of PIL.Image: List containing three sliced images.
    """

    # Compute the new size keeping the original aspect ratio
    old_size = im.size
    ratio = float(desired_size) / min(old_size)
    new_size = tuple([int(x * ratio) for x in old_size])

    # Resize the image
    im = im.resize(new_size, Image.ANTIALIAS)
    ar = np.array(im)

    images = []

    # Determine the middle of the image and half of the desired size
    half = desired_size // 2

    # Check if the image is landscape or portrait
    if ar.shape[0] < ar.shape[1]:  # Landscape
        middle = ar.shape[1] // 2
        images.append(Image.fromarray(ar[:, :desired_size]))
        images.append(Image.fromarray(ar[:, middle-half:middle+half]))
        images.append(Image.fromarray(ar[:, ar.shape[1]-desired_size:]))
    else:  # Portrait
        middle = ar.shape[0] // 2
        images.append(Image.fromarray(ar[:desired_size, :]))
        images.append(Image.fromarray(ar[middle-half:middle+half, :]))
        images.append(Image.fromarray(ar[ar.shape[0]-desired_size:, :]))

    return images


def resize_pad_image(im, desired_size):
    """
    Resize the input image and pad it to fit the desired size.

    Parameters:
    - im (PIL.Image): Input image.
    - desired_size (int): Desired size for the output image.

    Returns:
    - PIL.Image: Resized and padded image.
    """

    # Compute the new size keeping the original aspect ratio
    old_size = im.size
    ratio = float(desired_size) / max(old_size)
    new_size = tuple([int(x * ratio) for x in old_size])

    # Resize the image
    im = im.resize(new_size, Image.ANTIALIAS)

    # Create a new blank image and paste the resized image onto it
    new_im = Image.new("RGB", (desired_size, desired_size))
    new_im.paste(im, ((desired_size-new_size[0])//2, (desired_size-new_size[1])//2))

    return new_im

In [6]:
class ClipEncoderMulti(nn.Module):
    """
    CLIP Encoder to encode multiple images into fixed-size embeddings.

    Attributes:
    - model (nn.Module): CLIP model for image encoding.
    - num_embeds (int): Number of embeddings per input.
    - num_features (int): Size of the feature vector for each embedding.
    """

    def __init__(self, num_embeds, num_features=image_features_size):
        """
        Initialize the ClipEncoderMulti class.

        Parameters:
        - num_embeds (int): Number of embeddings per input.
        - num_features (int): Size of the feature vector for each embedding (default=image_features_size).
        """
        super(ClipEncoderMulti, self).__init__()
        self.model = clip_model
        self.num_embeds = num_embeds
        self.num_features = num_features

    def forward(self, x):
        """
        Forward pass of the ClipEncoderMulti model.

        Parameters:
        - x (torch.Tensor): Input tensor of shape [batch_size, num_embeds, channels, height, width].

        Returns:
        - torch.Tensor: Encoded images of shape [batch_size, num_embeds, num_features].
        """

        # Reshape input for encoding: merge the batch and num_embeds dimensions
        x_reshaped = x.view(-1, 3, 288, 288)

        # Encode each image and get the embeddings
        out = self.model.encode_image(x_reshaped)

        # Reshape output to separate batch size and num_embeds dimensions
        out = out.view(-1, self.num_embeds, self.num_features).float()

        return out


In [7]:
class JsonlDataset(Dataset):
    """
    Dataset class for loading data from a JSONL file format.

    Attributes:
    - data (List[Dict]): Loaded data from the JSONL file.
    - data_dir (str): Directory path where the data file is located.
    - tokenizer (Tokenizer): Tokenizer for text preprocessing.
    - max_seq_length (int): Maximum sequence length for the text.
    - transforms (callable): Transformations to apply on images.
    """

    def __init__(self, data_path, tokenizer, transforms, max_seq_length):
        """
        Initialize the JsonlDataset class.

        Args:
        - data_path (str): Path to the JSONL data file.
        - tokenizer (Tokenizer): Tokenizer for text preprocessing.
        - transforms (callable): Transformations to apply on images.
        - max_seq_length (int): Maximum sequence length for the text.
        """
        self.data = [json.loads(l) for l in open(data_path)]
        self.data_dir = os.path.dirname(data_path)
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length
        self.transforms = transforms

    def __len__(self):
        """Returns the total number of data samples."""
        return len(self.data)

    def __getitem__(self, index):
        """
        Fetches a single data sample.

        Args:
        - index (int): Index of the data sample.

        Returns:
        - Dict[Tensor]: A dictionary containing processed text and image data.
        """
        # Process text
        sentence = torch.LongTensor(self.tokenizer.encode(self.data[index]["text_with_tags"], add_special_tokens=True))
        start_token, sentence, end_token = sentence[0], sentence[1:-1], sentence[-1]
        sentence = sentence[:self.max_seq_length]
        label = torch.FloatTensor([self.data[index]["label"]])

        # Process image
        image = Image.open(os.path.join(self.data_dir, self.data[index]["img"])).convert("RGB")
        sliced_images = slice_image(image, 288)
        sliced_images = [np.array(self.transforms(im)) for im in sliced_images]
        image = resize_pad_image(image, image_encoder_size)
        image = np.array(self.transforms(image))
        sliced_images = [image] + sliced_images
        sliced_images = torch.from_numpy(np.array(sliced_images)).to(device)

        return {
            "image_start_token": start_token,
            "image_end_token": end_token,
            "sentence": sentence,
            "image": sliced_images,
            "label": label
        }

    def get_label_frequencies(self):
        """Returns a counter of label frequencies."""
        return Counter(row["label"] for row in self.data)

    def get_labels(self):
        """Returns a list of all labels in the dataset."""
        return [row["label"] for row in self.data]


def collate_fn(batch):
    """
    Collation function to process a batch of data samples.

    Args:
    - batch (List[Dict[Tensor]]): List of data samples.

    Returns:
    - Tuple[Tensor]: Processed batch data.
    """
    lens = [len(row["sentence"]) for row in batch]
    bsz, max_seq_len = len(batch), max(lens)

    # Initialize tensors
    mask_tensor = torch.zeros(bsz, max_seq_len, dtype=torch.long)
    text_tensor = torch.zeros(bsz, max_seq_len, dtype=torch.long)

    # Fill in data
    for i_batch, (input_row, length) in enumerate(zip(batch, lens)):
        text_tensor[i_batch, :length] = input_row["sentence"]
        mask_tensor[i_batch, :length] = 1

    # Extract other data features
    img_tensor = torch.stack([row["image"] for row in batch])
    tgt_tensor = torch.stack([row["label"] for row in batch])
    img_start_token = torch.stack([row["image_start_token"] for row in batch])
    img_end_token = torch.stack([row["image_end_token"] for row in batch])

    return text_tensor, mask_tensor, img_tensor, img_start_token, img_end_token, tgt_tensor


In [8]:
path = "/content/drive/MyDrive/cs688project2/data/"

In [9]:
df = pd.read_csv(path + 'final_cleaned_with_all_tags.csv')
df_train = df[:8500]
df_val = df[8500:9540]
df_test = df[9540:]

In [10]:
df_train.to_json('/content/drive/MyDrive/cs688project2/data/train_3.jsonl', orient='records', lines=True)
df_val.to_json('/content/drive/MyDrive/cs688project2/data/val_3.jsonl', orient='records', lines=True)

In [11]:
def load_examples(tokenizer, evaluate=False):
    """
    Load dataset examples from JSONL files.

    Args:
    - tokenizer (Tokenizer): Tokenizer for text preprocessing.
    - evaluate (bool, optional): Whether to load validation data or training data. Default: False (load training data).

    Returns:
    - JsonlDataset: Dataset containing loaded examples.
    """
    # Determine the path based on whether it's for evaluation or training.
    path = os.path.join(data_dir, "val_3.jsonl" if evaluate else "train_3.jsonl")
    # Instantiate the dataset using the given path, tokenizer, transforms, and a pre-defined max sequence length.
    dataset = JsonlDataset(path, tokenizer, preprocess, max_seq_length - num_image_embeds - 2)
    return dataset


def save_checkpoint(save_path, model, valid_loss):
    """
    Save the model's state along with its validation loss.

    Args:
    - save_path (str): Path where the model should be saved.
    - model (nn.Module): Model to save.
    - valid_loss (float): Validation loss associated with the model.

    Returns:
    - None
    """
    if not save_path:
        return

    # Define the state dictionary.
    state_dict = {
        'model_state_dict': model.state_dict(),
        'valid_loss': valid_loss
    }

    # Save the model.
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')


def load_checkpoint(load_path, model):
    """
    Load a model's state from a checkpoint.

    Args:
    - load_path (str): Path from where the model should be loaded.
    - model (nn.Module): Model to which the state should be loaded.

    Returns:
    - float: The validation loss associated with the loaded model state.
    """
    if not load_path:
        return

    # Load the state dictionary.
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')

    # Load the state into the model.
    model.load_state_dict(state_dict['model_state_dict'])
    return state_dict['valid_loss']


In [12]:
# Name of the pre-trained model
model_name = 'Hate-speech-CNERG/bert-base-uncased-hatexplain'

# Load configuration for the transformer model. This includes settings specific to the model, like number of layers and hidden sizes.
transformer_config = AutoConfig.from_pretrained(model_name)

# Load the pre-trained transformer model with the specified configuration
transformer = AutoModel.from_pretrained(model_name, config=transformer_config)

# Create an instance of the ClipEncoder for multi-modal (image + text) embeddings
img_encoder = ClipEncoderMulti(num_image_embeds)

# Initialize the tokenizer for the model. This will help in converting text into format suitable for model input.
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)

# Set configuration for the MMBT (Multi-Modal Bitransformers) model. It includes settings from transformer_config and additional parameters for multi-modality.
config = MMBTConfig(transformer_config, num_labels=num_labels, modal_hidden_size=image_features_size)

# Initialize the MMBT model for classification tasks, integrating the transformer and image encoder.
model = MMBTForClassification(config, transformer, img_encoder)

# Move the model to the specified device (e.g., GPU or CPU)
model.to(device);


In [13]:
# Load training examples from the data source using a custom function; 'evaluate' flag indicates if it's for evaluation or not.
train_dataset = load_examples(tokenizer, evaluate=False)

# Similarly, load evaluation examples. The 'evaluate' flag is set to True to differentiate it from the training dataset.
eval_dataset = load_examples(tokenizer, evaluate=True)

# Create a sampler for the training dataset that selects data randomly. This helps in stochastic gradient descent optimization.
train_sampler = RandomSampler(train_dataset)

# For the evaluation dataset, samples are drawn sequentially. This ensures we cover the entire dataset during evaluation.
eval_sampler = SequentialSampler(eval_dataset)

# Initialize a DataLoader for the training dataset.
# DataLoader facilitates efficient loading of data in batches during training.
# - 'sampler' is used to specify the sampling strategy.
# - 'batch_size' determines the number of examples in each batch.
# - 'collate_fn' is a function to collate/batch the data samples into a batched format.
train_dataloader = DataLoader(
        train_dataset,
        sampler=train_sampler,
        batch_size=train_batch_size,
        collate_fn=collate_fn
    )

# Similarly, initialize a DataLoader for the evaluation dataset.
eval_dataloader = DataLoader(
        eval_dataset,
        sampler=eval_sampler,
        batch_size=eval_batch_size,
        collate_fn=collate_fn
    )


In [14]:
# Prepare optimizer and its schedule (linear warmup and decay)

# List of model parameters that shouldn't undergo decay.
# These often include bias terms and normalization layer weights as decaying them might harm model's training.
no_decay = ["bias", "LayerNorm.weight"]

# Set the default weight decay. Weight decay is a form of regularization to prevent overfitting.
weight_decay = 0.0005

# Group model parameters based on whether they should undergo weight decay or not.
optimizer_grouped_parameters = [
        {
            # Parameters which should have weight decay applied
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": weight_decay,
        },
        {
            # Parameters which shouldn't have weight decay applied
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]

# Calculate the total number of training steps. This is used for scheduler.
# gradient_accumulation_steps refers to the number of steps to accumulate gradients before optimizing.
t_total = (len(train_dataloader) // gradient_accumulation_steps) * num_train_epochs

# Define warmup steps as a fraction of total training steps.
# Warmup is a period where the learning rate is gradually increased from a very small value to the set value.
warmup_steps = t_total // 10

# Initialize the MADGRAD optimizer with the specified parameters and learning rate.
optimizer = MADGRAD(optimizer_grouped_parameters, lr=2e-4)

# Set up the learning rate scheduler with linear warmup at the beginning and decay afterwards.
scheduler = get_linear_schedule_with_warmup(
        optimizer, warmup_steps, t_total
    )

# Define the loss function for training.
# BCEWithLogitsLoss combines a Sigmoid layer and the BCELoss (Binary Cross-Entropy loss) in one single class.
criterion = nn.BCEWithLogitsLoss()


In [15]:
def evaluate(model, tokenizer, criterion, dataloader, threshold=0.5):
    """
    Evaluate the model on a given dataloader.

    Parameters:
    - model: The model to be evaluated.
    - tokenizer: Tokenizer for text preprocessing.
    - criterion: The loss function.
    - dataloader: DataLoader for evaluation data.
    - threshold: Threshold for classification, defaults to 0.5.

    Returns:
    - result: Dictionary containing evaluation metrics and other details.
    """

    # Initialize variables for evaluation metrics and predictions.
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = []
    proba = []
    out_label_ids = []

    # Set the model in evaluation mode.
    model.eval()

    # Loop over batches from the dataloader.
    for batch in dataloader:
        # Move batch tensors to the same device as the model.
        batch = tuple(t.to(device) for t in batch)

        # Disable gradient calculations, as they aren't needed for evaluation.
        with torch.no_grad():
            labels = batch[5]
            inputs = {
                "input_ids": batch[0],
                "input_modal": batch[2],
                "attention_mask": batch[1],
                "modal_start_tokens": batch[3],
                "modal_end_tokens": batch[4],
                "return_dict": False
            }

            # Obtain model outputs.
            outputs = model(**inputs)
            logits = outputs[0]  # Extract logits from outputs.

            # Calculate loss for the current batch.
            tmp_eval_loss = criterion(logits, labels)
            eval_loss += tmp_eval_loss.mean().item()

        nb_eval_steps += 1

        # Convert logits to probabilities and predictions.
        batch_proba = torch.sigmoid(logits).detach().cpu().numpy()
        batch_preds = batch_proba > threshold
        batch_labels = labels.detach().cpu().numpy()

        # Append batch results to lists.
        preds.extend(batch_preds)
        proba.extend(batch_proba)
        out_label_ids.extend(batch_labels)

    # Calculate the average loss over all batches.
    eval_loss = eval_loss / nb_eval_steps

    # Create a result dictionary with evaluation metrics.
    result = {
        "loss": eval_loss,
        "accuracy": accuracy_score(out_label_ids, preds),
        "AUC": roc_auc_score(out_label_ids, proba),
        "micro_f1": f1_score(out_label_ids, preds, average="micro"),
        "prediction": np.array(preds),
        "labels": np.array(out_label_ids),
        "proba": np.array(proba)
    }

    return result


In [16]:
# Initialization of variables for training metrics, model state tracking and checkpointing.
optimizer_step = 0
global_step = 0
train_step = 0
tr_loss, logging_loss = 0.0, 0.0
best_valid_auc = 0.75
global_steps_list = []
train_loss_list = []
val_loss_list = []
val_acc_list = []
val_auc_list = []
eval_every = len(train_dataloader) // 7  # Frequency of evaluation during training.
running_loss = 0
checkpoint_directory = "/content/drive/MyDrive/cs688project2/data/models/"

# Zero out model gradients at the start.
model.zero_grad()

# Training loop for multiple epochs.
for epoch in range(num_train_epochs):
    print(f"Epoch {epoch + 1} of {num_train_epochs}")

    # Tracking predictions and true labels for metrics calculation.
    whole_y_pred = np.array([])
    whole_y_t = np.array([])

    # Loop through each batch from the training dataloader.
    for step, batch in enumerate(tqdm(train_dataloader)):
        # Set model to training mode.
        model.train()

        # Transfer batch to device.
        batch = tuple(t.to(device) for t in batch)
        labels = batch[5]

        # Prepare inputs for the model.
        inputs = {
            "input_ids": batch[0],
            "input_modal": batch[2],
            "attention_mask": batch[1],
            "modal_start_tokens": batch[3],
            "modal_end_tokens": batch[4],
            "return_dict": False
        }

        # Get model outputs.
        outputs = model(**inputs)
        logits = outputs[0]
        loss = criterion(logits, labels)

        # Handle gradient accumulation if set.
        if gradient_accumulation_steps > 1:
            loss = loss / gradient_accumulation_steps

        loss.backward()

        # Update training loss trackers.
        tr_loss += loss.item()
        running_loss += loss.item()
        global_step += 1

        # Update model parameters if gradient accumulation is fulfilled.
        if (step + 1) % gradient_accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            scheduler.step()  # Update learning rate schedule
            optimizer_step += 1
            optimizer.zero_grad()

        # Evaluate model periodically.
        if (step + 1) % eval_every == 0:
            average_train_loss = running_loss / eval_every
            train_loss_list.append(average_train_loss)
            global_steps_list.append(global_step)
            running_loss = 0.0

            # Validate the model.
            val_result = evaluate(model, tokenizer, criterion, eval_dataloader)
            val_loss_list.append(val_result['loss'])
            val_acc_list.append(val_result['accuracy'])
            val_auc_list.append(val_result['AUC'])

            # Save model checkpoint if AUC has improved.
            if val_result['AUC'] > best_valid_auc:
                best_valid_auc = val_result['AUC']
                model_path = f'{checkpoint_directory}/model-embs{num_image_embeds}-seq{max_seq_length}-auc{best_valid_auc:.3f}-loss{val_result["loss"]:.3f}-acc{val_result["accuracy"]:.3f}.pt'
                print(f"AUC improved, so saving this model to {model_path}")
                save_checkpoint(model_path, model, val_result['loss'])

            # Log training and validation metrics.
            print(f"Train loss: {average_train_loss:.4f}, Val loss: {val_result['loss']:.4f}, Val accuracy: {val_result['accuracy']:.4f}, AUC: {val_result['AUC']:.4f}")

    print('\n')


Epoch 1 of 5


  0%|          | 0/532 [00:00<?, ?it/s]

<ipython-input-5-f96d43e93617>:19: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  im = im.resize(new_size, Image.ANTIALIAS)
<ipython-input-5-f96d43e93617>:60: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  im = im.resize(new_size, Image.ANTIALIAS)


Train loss: 0.0327, Val loss: 0.7610, Val accuracy: 0.5000, AUC: 0.5270


<ipython-input-5-f96d43e93617>:19: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  im = im.resize(new_size, Image.ANTIALIAS)
<ipython-input-5-f96d43e93617>:60: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  im = im.resize(new_size, Image.ANTIALIAS)


...

  0%|          | 0/532 [00:00<?, ?it/s]

<ipython-input-5-f96d43e93617>:19: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  im = im.resize(new_size, Image.ANTIALIAS)
<ipython-input-5-f96d43e93617>:60: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  im = im.resize(new_size, Image.ANTIALIAS)


Train loss: 0.0241, Val loss: 0.6834, Val accuracy: 0.6404, AUC: 0.7070


<ipython-input-5-f96d43e93617>:19: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  im = im.resize(new_size, Image.ANTIALIAS)
<ipython-input-5-f96d43e93617>:60: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  im = im.resize(new_size, Image.ANTIALIAS)


KeyboardInterrupt: ignored